# The `open` event

The `open` audit event is raised when a file is opened by the `open` builtin and some other mechanisms.

In [1]:
import pathlib

import subaudit

## The `open` builtin

In [2]:
with subaudit.extracting('open', lambda *args: (*args,)) as extracts:
    with open('../LICENSE', encoding='utf-8'):
        pass

extracts

[('../LICENSE', 'r', 524288)]

On CPython, the above output tends to look like:

```python
[('../README.md', 'r', 524288)]
```

On PyPy, it tends to look like:

```python
[('../LICENSE', 'r', 1)]
```

## `pathlib.Path`’s `read_*` and `write_*` methods

**NOTE: This section contains substantial *wrong* information, which *must* be
corrected before this is merged to the main branch!**

Using a `pathlib.Path` object to open a file, by calling its `read_bytes`,
`read_text`, `write_bytes`, or `write_text` methods, likewise raises the `open`
event, as if (and probably due to) their use of `open`.

In CPython, a call to one of those methods raises the `open` event only once.

In PyPy, a call to one of those methods raises the `open` event *twice*: the
method raises it on behalf of itself as well (rather than just the `open`
builtin or however else the behavior is implemented). This can be distinguished
because it passes a `Path` object, rather than a `str`, for the `path`
argument, and passes `None` for the `mode` argument. This additional event is
raised *after* the regular one.

In [3]:
with subaudit.extracting('open', lambda *args: (*args,)) as extracts:
    pathlib.Path('../README.md').read_text(encoding='utf-8')

extracts

[('../README.md', 'r', 524288), ('../README.md', None, 524288)]

On CPython, the above output tends to look like:

```python
[('../README.md', 'r', 524288)]
```

On PyPy, it tends to look like:

```python
[('../README.md', 'r', 1), (PosixPath('../README.md'), None, 524288)]
```